In [1]:
import heapq
import math
import os
import re

import aocd
import numpy as np
from IPython.display import HTML
from scipy.ndimage import convolve

In [2]:
p = aocd.get_puzzle(year=2025, day=6)

In [3]:
p.examples[0].input_data.split("\n")

['123 328  51 64 ', ' 45 64  387 23 ', '  6 98  215 314', '*   +   *   +  ']

In [4]:
def get_data(test_data: bool = False):
    if test_data:
        data = p.examples[0].input_data
    else:
        data = p.input_data
    return data

## Part 1

In [5]:
def process_data(data):
    grid = [l.split() for l in data.split("\n")]
    operators = grid[-1]
    numbers = np.array(grid[:-1]).T
    return operators, numbers

In [6]:
data = get_data(test_data=False)
operators, numbers = process_data(data)

In [7]:
%%time
# Part 1
res = 0

for operator, num in zip(operators, numbers):
    match operator:
        case "*":
            res += math.prod(num.astype(int))
        case "+":
            res += np.sum(num.astype(int))

res

CPU times: user 2.43 ms, sys: 4 μs, total: 2.44 ms
Wall time: 2.44 ms


7229350537438

## Part 2

In [8]:
def process_data2(data):
    grid = np.array([list(l) for l in data.split("\n")])
    return grid.T

In [9]:
data = get_data(test_data=False)
grid = process_data2(data)

In [10]:
%%time

total = 0

operator = None
OPERATORS = ("*", "+")

for row in grid:
    line = "".join(row)

    if not line.strip():
        # Empty row initiat and move to next
        total += res
        res = 0
        continue

    last_char = line[-1]
    number = int("".join(line[:-1]))

    if last_char in OPERATORS:
        operator = last_char
        res = number
    elif operator == "*":
        res *= number
    elif operator == "+":
        res += number

# Add the last result to total
total += res

total

CPU times: user 5.29 ms, sys: 45 μs, total: 5.34 ms
Wall time: 5.34 ms


11479269003550

## Visualization

In [13]:
import time

from IPython.display import HTML, clear_output, display

total = 0
res = 0
operator = None
OPERATORS = ("*", "+")


def get_html(preview_rows, i):
    return f"""
    <div style="font-family:monospace;padding:10px;background:#1e1e1e;color:#fff;border-radius:8px;">
        <div style="display:flex;gap:20px;margin-bottom:10px;">
            <div style="background:#2d2d2d;padding:8px 15px;border-radius:5px;min-width:140px;">
                <span style="color:#888;">Row</span> 
                <span style="color:#4fc3f7;font-size:1.2em;">{i+1}</span>
                <span style="color:#666;">/ {grid.shape[0]}</span>
            </div>
            <div style="background:#2d2d2d;padding:8px 15px;border-radius:5px;min-width:200px;">
                <span style="color:#888;">Total:</span> 
                <span style="color:#4caf50;font-size:1.2em;">{total:>12}</span>
            </div>
            <div style="background:#2d2d2d;padding:8px 15px;border-radius:5px;min-width:220px;">
                <span style="color:#888;">Sub-total:</span> 
                <span style="color:#ff9800;font-size:1.2em;">{res:>12}</span>
            </div>
            <div style="background:#2d2d2d;padding:8px 15px;border-radius:5px;min-width:100px;">
                <span style="color:#888;">Operator:</span> 
                <span style="color:{op_color};font-size:1.3em;font-weight:bold;">{op_display}</span>
            </div>
        </div>
        <div style="background:#252525;padding:10px;border-radius:5px;font-size:12px;line-height:1.4;">
            {preview_rows}
        </div>
        <div style="margin-top:8px;background:#333;border-radius:3px;overflow:hidden;">
            <div style="width:{100*i/grid.shape[0]:.1f}%;height:4px;background:linear-gradient(90deg,#4caf50,#8bc34a);"></div>
        </div>
    </div>
    """


for i, row in enumerate(grid):
    line = "".join(row)

    # Build the preview rows with current row highlighted
    preview_rows = ""
    for j in range(15):
        idx = i + j
        if idx < grid.shape[0]:
            row_text = (
                "".join(grid[idx][:-1]).replace(" ", "&nbsp;") + "  " + grid[idx][-1]
            )
            if j == 0:
                preview_rows += f'<div style="border-left:3px solid #ff9800;padding-left:8px;font-weight:bold;">▶ {res}</div>'
            else:
                preview_rows += f'<div style="color:#666;">&nbsp; {row_text}</div>'

    # Operator display
    op_display = operator if operator else "—"
    op_color = (
        "#e74c3c" if operator == "*" else "#3498db" if operator == "+" else "#999"
    )

    if not line.strip():
        total += res
        clear_output(wait=True)
        display(HTML(get_html(preview_rows, i)))
        res = 0
        time.sleep(0.01)
        continue

    last_char = line[-1]
    number = int("".join(line[:-1]))

    if last_char in OPERATORS:
        operator = last_char
        res = number
    elif operator == "*":
        res *= number
    elif operator == "+":
        res += number

total += res
clear_output(wait=True)
display(HTML(get_html(preview_rows, i)))
total

11479269003550